In [2]:
library(feather)
library(varbvs)

In [3]:
get_rds_toy <- function(datalist){
    for (dat in datalist){
        newdat = read_feather(dat)
        newmatrix = matrix(as.double(unlist(newdat)), dim(newdat)[1], dim(newdat)[2])
        colnames(newmatrix) = colnames(newdat)
        newfit = varbvs(newmatrix[,-1], NULL, newmatrix[,1], family = "binomial", 
                        logodds = seq(-3.5,-1,0.1), sa = 1, verbose = F)
        newfit$summary = summary(newfit)
        newfit$w = normalizelogweights(newfit$logw)
        newfit$pip = newfit$alpha %*% c(newfit$w)
        filename = paste(dat, ".RDS", sep="")
        saveRDS(newfit, filename)
        return (newfit$summary$top.vars)
    }
}

In [4]:
# get_rds_toy(c("data/toy_n000_600_n100_100_n010_100_n001_100_n110_100_n011_100_n111_100.feather"))
get_rds_toy(c("data/toy_4genes_n1320_const0_causal1.feather"))

index,variable,prob,PVE,coef,Pr(coef.>0.95)
2,gene2,0.002438732,NA,-0.08537077,"[-0.284,+0.130]"
1,gene1,0.002415618,NA,-0.08443071,"[-0.317,+0.131]"
3,gene3,0.002001100,NA,0.04889482,"[-0.164,+0.262]"
4,gene4,0.001847391,NA,0.02403800,"[-0.190,+0.221]"


In [7]:
newdat = read_feather("data/toy_4genes_n_1440.feather")
newmatrix = matrix(as.double(unlist(newdat)), dim(newdat)[1], dim(newdat)[2])
colnames(newmatrix) = colnames(newdat)
# print (dim(newdat)[1])
newfit = varbvs(newmatrix[,-1], NULL, newmatrix[,1], family = "binomial", 
                logodds = seq(-3.5,-1,0.1), sa = 1, verbose = F)
newfit$summary = summary(newfit)
newfit$w = normalizelogweights(newfit$logw)
newfit$pip = newfit$alpha %*% c(newfit$w)
newfit$beta = newfit$mu %*% c(newfit$w)
newfit$var_names = colnames(newdat[-1])
print (summary(newfit))
dimnames(newfit$pip)[[1]]

Summary of fitted Bayesian variable selection model:
family:     binomial   num. hyperparameter settings: 26
samples:    1440       iid variable selection prior: yes
variables:  4          fit prior var. of coefs (sa): no
covariates: 1          fit approx. factors (eta):    yes
maximum log-likelihood lower bound: -971.6803
Hyperparameters: 
        estimate Pr>0.95             candidate values
sa            NA [NA,NA]             1--1
logodds    -1.27 [-2.00,-1.00]       (-3.50)--(-1.00)
Selected variables by probability cutoff:
>0.10 >0.25 >0.50 >0.75 >0.90 >0.95 
    2     2     2     1     1     1 
Top 4 variables by inclusion probability:
  index variable   prob PVE  coef*  Pr(coef.>0.95)
1     1    gene1 1.0000  NA  0.870 [+0.659,+1.087]
2     3    gene3 0.5730  NA -0.353 [-0.571,-0.153]
3     2    gene2 0.0662  NA -0.238 [-0.440,-0.025]
4     4    gene4 0.0282  NA -0.188 [-0.397,+0.005]
*See help(varbvs) about interpreting coefficients in logistic regression.


[1] "gene1" "gene2" "gene3" "gene4"